In [7]:
import re
import stanza
from joblib import dump, load
from sklearn.feature_extraction.text import TfidfVectorizer
from fastapi import FastAPI
from nltk.corpus import stopwords

stanza.download('es') 
# stopwords.words("spanish")
nlp = stanza.Pipeline('es', processors='tokenize,mwt,pos,lemma')

url = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
menciones = '@[\w\-]+'
hashtag = '#[\w\-]+'
caracteres_especiales = r'\W'
varios_espacios= r'\s+'

def clean_tweet(tweet):
    tweet_procesado = tweet.lower() 
    tweet_procesado = re.sub(menciones, ' ', tweet_procesado)
    tweet_procesado = re.sub(url, ' ', tweet_procesado)
    tweet_procesado = re.sub(caracteres_especiales, ' ', tweet_procesado)
    tweet_procesado = re.sub(" q ", ' que ', tweet_procesado)
    tweet_procesado = re.sub(" pq ", ' porque ', tweet_procesado)
    tweet_procesado = re.sub(" xq ", ' porque ', tweet_procesado)
    tweet_procesado = re.sub(varios_espacios, ' ', tweet_procesado, flags=re.I)
    return tweet_procesado

def procesar_texto(cadena):
    palabras = cadena.split() 
    cadena_limpia = [palabra for palabra in palabras if palabra.lower() not in stopwords.words("spanish")]
    return cadena_limpia

SVM = load('support-vector-machine.joblib')
Naive = load('naive-bayes-classifier.joblib')
RFC = load('random-forest-classifier.joblib')
tfidf_vector = load('tfidf_vector.joblib')

def predict_mental_issues(tweet, model):
    cleaned = clean_tweet(tweet)
    procesado = procesar_texto(cleaned)
    texto_procesado = ' '.join(procesado)
    doc = nlp(texto_procesado)
    lemmas = []
    for sent in doc.sentences :
        for word in sent.words:
            lemmas.append(word.lemma)
    print(lemmas)
    texto_final = ' '.join(lemmas)
    Test_X_Tfidf = tfidf_vector.transform([texto_final])
    print(Test_X_Tfidf)
    if model == 'NB':
        predictions = Naive.predict(Test_X_Tfidf)
    elif model == 'SVM':
        predictions = SVM.predict(Test_X_Tfidf)
    elif model == 'RFC':
        predictions = RFC.predict(Test_X_Tfidf)
    else:
        predictions = 'No se encontro el modelo'
    return predictions

2022-01-20 18:26:45 INFO: Downloading default packages for language: es (Spanish)...
2022-01-20 18:26:48 INFO: File exists: C:\Users\USUARIO\stanza_resources\es\default.zip.
2022-01-20 18:26:51 INFO: Finished downloading models and saved to C:\Users\USUARIO\stanza_resources.
2022-01-20 18:26:51 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |

2022-01-20 18:26:51 INFO: Use device: cpu
2022-01-20 18:26:51 INFO: Loading: tokenize
2022-01-20 18:26:51 INFO: Loading: mwt
2022-01-20 18:26:51 INFO: Loading: pos
2022-01-20 18:26:51 INFO: Loading: lemma
2022-01-20 18:26:51 INFO: Done loading processors!


In [8]:
app = FastAPI()

@app.post("/mentalhealth")
def analyze_tweet(tweet: str, model: str):
    try:
        prediccion = predict_mental_issues(tweet, 'SVM')
    except Exception as e:
        print(e)
    return {'prediccion': prediccion[0]}

In [9]:
!cd ./ && uvicorn main:app

Traceback (most recent call last):
  File "C:\Users\USUARIO\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\USUARIO\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\USUARIO\AppData\Local\Programs\Python\Python310\Scripts\uvicorn.exe\__main__.py", line 7, in <module>
  File "C:\Users\USUARIO\AppData\Local\Programs\Python\Python310\lib\site-packages\click\core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "C:\Users\USUARIO\AppData\Local\Programs\Python\Python310\lib\site-packages\click\core.py", line 782, in main
    rv = self.invoke(ctx)
  File "C:\Users\USUARIO\AppData\Local\Programs\Python\Python310\lib\site-packages\click\core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "C:\Users\USUARIO\AppData\Local\Programs\Python\Python310\lib\site-packages\